## 3D Terzaghi Consolidation Problem (PINN)

Here, we attempt to invert all *three* parameters from the 3D Terzaghi PDE by using the physics to predict the unknown parameters from exact solution data. The real parameter values are 0.05, 0.10, 0.15 respectively. The code is incomplete as an exact solution of 60x60x60 grid requires > 100GB data. The result for a single parameter is available in the results folder.

In [ ]:
import numpy as np 
import sciann as sn 
import pandas as pd
import matplotlib.pyplot as plt

from sciann.utils.math import sign, abs, sigmoid, tanh, diff

In [ ]:
x_min, x_max = 0, 1
y_min, y_max = 0, 1
z_min, z_max = 0, 1
t_min, t_max = 0, 1

N = 30

xs = np.linspace(x_min, x_max, N)
ys = np.linspace(y_min, y_max, N)
zs = np.linspace(z_min, z_max, N)
ts = np.linspace(t_min, t_max, 21600)

In [ ]:
def PrepareData(num_data=50000, random=True):
    # Flattening the mesh grid
    X, Y, Z, T = np.meshgrid(xs, ys, zs, ts)
    X_flat = X.flatten()[:, None]
    Y_flat = Y.flatten()[:, None]
    Z_flat = Z.flatten()[:, None]
    T_flat = T.flatten()[:, None]
    
    # Calculating U_mesh
    U_mesh_flat = U_mesh.flatten()[:, None]
    
    # Pick random data
    if random:
        idx = np.random.choice(Z_flat.shape[0], num_data, replace=False)
    else:
        idx = np.arange(0, Z_flat.shape[0])
    
    x_train = X_flat[idx]
    y_train = Y_flat[idx]
    z_train = Z_flat[idx]
    t_train = T_flat[idx]
    u_train = U_mesh_flat[idx]
    
    return (x_train, y_train, z_train, t_train, u_train)

x_train, y_train, z_train, t_train, u_train = PrepareData(50000, random=True)

In [ ]:
# Define independent variables
x = sn.Variable("x", dtype='float64')
y = sn.Variable("y", dtype='float64')
z = sn.Variable("z", dtype='float64')
t = sn.Variable("t", dtype='float64')

# Define solution variable with z
U = sn.Functional("U", [x, y, z, t], 4*[40], 'tanh')

# Define PDE (physics constraint)
U_t = diff(U, t)
U_x = diff(U, x)
U_y = diff(U, y)
U_z = diff(U, z)
U_xx = diff(U_x, x)
U_yy = diff(U_y, y)
U_zz = diff(U_z, z)

single = False

# Define parameters to be inverted
if single:
    cv_inv = sn.Parameter(2.0, non_neg=True, inputs=[x, y, z, t], name="cv_inv")
    
    PDE = cv_inv*(U_xx + U_yy + U_zz) - U_t
    
else:
    cx_inv = sn.Parameter(0.5, min_max = [0.01, cx*1.25], non_neg=True, inputs=[x, y, z, t], name="cx_inv")
    cy_inv = sn.Parameter(0.5, min_max = [0.01, cy*1.25], non_neg=True, inputs=[x, y, z, t], name="cy_inv")
    cz_inv = sn.Parameter(0.5, min_max = [0.01, cz*1.25], non_neg=True, inputs=[x, y, z, t], name="cz_inv")
    
    PDE = cx_inv*U_xx + cy_inv*U_yy + cz_inv*U_zz - U_t


# Define targets (losses) using sn.Data and sn.PDE interfaces
data_target = sn.Data(U)
pde_constraint = sn.PDE(PDE)

# Collect all targets
targets = [data_target, pde_constraint]

input_data = [x_train, y_train, z_train, t_train]
target_data = [u_train, 'zeros']

In [ ]:
epochs = 25000
batch_size = 5000

model_inv = sn.SciModel(
    inputs=[x, y, z, t],
    targets=targets,
    loss_func="mse"
)

if single:
    params = {'parameters': cv_inv, 'freq': 1}

else:
    params = {'parameters': [cx_inv, cy_inv, cz_inv], 'freq': 1}

# Prepare the training input and target data
H_new = model_inv.train(
    input_data,
    target_data,
    epochs=epochs,
    batch_size=batch_size,
    stop_loss_value=1e-36,
    learning_rate=([0, 1000], [1e-2, 1e-3]),
    stop_after=None,
    verbose=2,
    log_parameters=params
)